In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
import pandas as pd


In [ ]:
df = pd.read_csv('matched_distinct.csv')

df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,msd_track_id
0,0,0,98,7DYsBLdOqz0z14tYWMt2Tn,KT Tunstall,Del gusto de mamá,Hold On,0,177613,False,...,0,0.0979,0.32200,0.000001,0.1900,0.805,105.549,4,acoustic,TRBTWXE128F933EF94
1,9,9,99,7cPuE0M35EajWPRO3nRcH8,KT Tunstall,OO's Music Grandi Successi,Suddenly I See,0,199040,False,...,1,0.0438,0.13900,0.000000,0.1640,0.693,100.499,4,acoustic,TRFPXXU128F933D7EB
2,27,27,137,5p9XWUdvbUzmPCukOmwoU3,KT Tunstall,Eye To The Telescope,Suddenly I See,73,201706,False,...,1,0.0449,0.22500,0.000000,0.1120,0.664,100.380,4,acoustic,TRFPXXU128F933D7EB
3,45,45,152,5aDpULK8MbJmHl42kR5KNI,Howie Day,Stop All The World Now,Collide,56,249120,False,...,1,0.0277,0.22200,0.000053,0.1190,0.342,93.931,4,acoustic,TROHRUF128F42AD54A
4,46,46,169,4E6cwWJWZw2zWf7VFbH7wf,Sara Bareilles,Little Voice,Love Song,73,258826,False,...,0,0.0301,0.02080,0.000000,0.1880,0.573,123.055,4,acoustic,TRKPRLO128F426624C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,7722,7722,111500,0QKQfBRO7yv8XCefwEax14,Lamb,What Sound,Gabriel,51,260666,False,...,0,0.0333,0.18000,0.006200,0.0562,0.187,83.718,4,trip-hop,TRAWEKG128F42B6303
1244,7724,7724,111517,1BQlFMbF8D0V0gutMmeSaA,Flunk,Morning Star,Blind My Mind,47,276533,False,...,1,0.0245,0.20700,0.061300,0.1390,0.209,100.041,4,trip-hop,TRWSRFF12903CA6410
1245,7725,7725,111569,7xgfSlAPUouBAquQZkAUdq,Morcheeba,Big Calm,Blindfold,52,278679,False,...,1,0.0348,0.02810,0.027400,0.0555,0.215,161.949,4,trip-hop,TRTCMKQ128E079654D
1246,7726,7726,111675,1nYuroLdp35I01UFOvQABT,Morcheeba,Fragments of Freedom,Be Yourself,52,194519,False,...,0,0.0283,0.10100,0.010300,0.1150,0.893,110.053,4,trip-hop,TRHEOIY128E079654E


In [ ]:
df.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'track_id', 'artists',
       'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'track_genre', 'msd_track_id'],
      dtype='object')

In [ ]:
df_selected = df[['msd_track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'acousticness', 'instrumentalness', 'valence', 'tempo', 'time_signature']]

In [ ]:
!unzip -qq results.zip

In [ ]:
import glob
import json
import pickle
import ast

descriptions = []
max_description_length = 0
for file_path in glob.glob('results/segments/*.json'):
  file_name = file_path.split('/')[-1]
  with open(file_path, mode='r') as f:
    description = ast.literal_eval(f.read())
  track_id = file_name.split('_')[0]
  descriptions.append((track_id, description))
  max_description_length = max(max_description_length, len(description))
  # descriptions.append((track_id, " ".join(description)))

print("Max description length", max_description_length)

df2 = pd.DataFrame(descriptions, columns=('msd_track_id', 'description'))

df2

Max description length 322


,msd_track_id,description
0,TRLOBFL128F4278F0D,"[Bar_61, Time Signature_4/4, Note Density_4, M..."
1,TROGWMB128E07928B9,"[Bar_85, Time Signature_4/4, Note Density_4, M..."
2,TRNRHQD128F427E465,"[Bar_49, Time Signature_2/4, Note Density_3, M..."
3,TRJQIVI12903CBB562,"[Bar_1, Time Signature_4/4, Note Density_1, Me..."
4,TRQIAHV128F92F53B8,"[Bar_73, Time Signature_4/4, Note Density_1, M..."
...,...,...
7090,TRFBKPC128EF3436D4,"[Bar_109, Time Signature_4/4, Note Density_10,..."
7091,TRCUNLB128E0783EC3,"[Bar_97, Time Signature_4/4, Note Density_1, M..."
7092,TRHCJMM128F933BEED,"[Bar_109, Time Signature_4/4, Note Density_3, ..."
7093,TRLTOOL128F92CFC61,"[Bar_97, Time Signature_4/4, Note Density_2, M..."


In [ ]:
df_merged = pd.merge(df2, df_selected, how='left', on='msd_track_id')

df_merged

,msd_track_id,description,danceability,energy,key,loudness,mode,acousticness,instrumentalness,valence,tempo,time_signature
0,TRLOBFL128F4278F0D,"[Bar_61, Time Signature_4/4, Note Density_4, M...",0.662,0.798,8,-4.627,1,0.3060,0.000000,0.542,141.007,4
1,TROGWMB128E07928B9,"[Bar_85, Time Signature_4/4, Note Density_4, M...",0.346,0.641,7,-5.627,1,0.1360,0.000008,0.159,77.281,4
2,TRNRHQD128F427E465,"[Bar_49, Time Signature_2/4, Note Density_3, M...",0.209,0.795,0,-7.241,1,0.0109,0.000056,0.682,171.975,4
3,TRJQIVI12903CBB562,"[Bar_1, Time Signature_4/4, Note Density_1, Me...",0.470,0.796,8,-3.277,0,0.3620,0.000001,0.215,113.108,3
4,TRQIAHV128F92F53B8,"[Bar_73, Time Signature_4/4, Note Density_1, M...",0.307,0.748,2,-6.514,0,0.2120,0.000153,0.347,108.227,4
...,...,...,...,...,...,...,...,...,...,...,...,...
10737,TRHCJMM128F933BEED,"[Bar_109, Time Signature_4/4, Note Density_3, ...",0.721,0.672,6,-9.029,0,0.0547,0.010600,0.804,104.976,4
10738,TRHCJMM128F933BEED,"[Bar_109, Time Signature_4/4, Note Density_3, ...",0.706,0.747,6,-6.336,0,0.0724,0.001150,0.800,104.960,4
10739,TRLTOOL128F92CFC61,"[Bar_97, Time Signature_4/4, Note Density_2, M...",0.442,0.885,4,-5.843,1,0.0810,0.000000,0.268,129.421,4
10740,TRAUWUX128F92FE41C,"[Bar_49, Time Signature_4/4, Note Density_3, M...",0.514,0.243,3,-12.348,1,0.7520,0.000540,0.388,171.816,4


In [ ]:
def extract_token_class(row, token_class):
  filtered_description = [
      token
      for token in row['description']
      if token_class in token
  ]
  return filtered_description

def extract_categorical_tokens(row):
  filtered_description = [
      token
      for token in row['description']
      if "Chord" in token or "Instrument" in token or "Time" in token or "Bar" in token
  ]
  return filtered_description

In [ ]:
df_merged['pitch_description'] = df_merged.apply(lambda row: extract_token_class(row, 'Mean Pitch'), axis=1)
df_merged['velocity_description'] = df_merged.apply(lambda row: extract_token_class(row, 'Mean Velocity'), axis=1)
df_merged['density_description'] = df_merged.apply(lambda row: extract_token_class(row, 'Density'), axis=1)
df_merged['duration_description'] = df_merged.apply(lambda row: extract_token_class(row, 'Mean Duration'), axis=1)
df_merged['categorical_description'] = df_merged.apply(extract_categorical_tokens, axis=1)
df_merged

,msd_track_id,description,danceability,energy,key,loudness,mode,acousticness,instrumentalness,valence,tempo,time_signature,pitch_description,velocity_description,density_description,duration_description,categorical_description
0,TRLOBFL128F4278F0D,"[Bar_61, Time Signature_4/4, Note Density_4, M...",0.662,0.798,8,-4.627,1,0.3060,0.000000,0.542,141.007,4,"[Mean Pitch_13, Mean Pitch_14, Mean Pitch_13, ...","[Mean Velocity_22, Mean Velocity_23, Mean Velo...","[Note Density_4, Note Density_5, Note Density_...","[Mean Duration_32, Mean Duration_32, Mean Dura...","[Bar_61, Time Signature_4/4, Instrument_Electr..."
1,TROGWMB128E07928B9,"[Bar_85, Time Signature_4/4, Note Density_4, M...",0.346,0.641,7,-5.627,1,0.1360,0.000008,0.159,77.281,4,"[Mean Pitch_14, Mean Pitch_14, Mean Pitch_14, ...","[Mean Velocity_19, Mean Velocity_19, Mean Velo...","[Note Density_4, Note Density_4, Note Density_...","[Mean Duration_32, Mean Duration_32, Mean Dura...","[Bar_85, Time Signature_4/4, Instrument_Synth ..."
2,TRNRHQD128F427E465,"[Bar_49, Time Signature_2/4, Note Density_3, M...",0.209,0.795,0,-7.241,1,0.0109,0.000056,0.682,171.975,4,"[Mean Pitch_13, Mean Pitch_13, Mean Pitch_14, ...","[Mean Velocity_20, Mean Velocity_20, Mean Velo...","[Note Density_3, Note Density_4, Note Density_...","[Mean Duration_29, Mean Duration_31, Mean Dura...","[Bar_49, Time Signature_2/4, Instrument_Electr..."
3,TRJQIVI12903CBB562,"[Bar_1, Time Signature_4/4, Note Density_1, Me...",0.470,0.796,8,-3.277,0,0.3620,0.000001,0.215,113.108,3,"[Mean Pitch_13, Mean Pitch_13, Mean Pitch_13, ...","[Mean Velocity_23, Mean Velocity_23, Mean Velo...","[Note Density_1, Note Density_1, Note Density_...","[Mean Duration_32, Mean Duration_32, Mean Dura...","[Bar_1, Time Signature_4/4, Instrument_Electri..."
4,TRQIAHV128F92F53B8,"[Bar_73, Time Signature_4/4, Note Density_1, M...",0.307,0.748,2,-6.514,0,0.2120,0.000153,0.347,108.227,4,"[Mean Pitch_16, Mean Pitch_14, Mean Pitch_14, ...","[Mean Velocity_26, Mean Velocity_27, Mean Velo...","[Note Density_1, Note Density_1, Note Density_...","[Mean Duration_32, Mean Duration_32, Mean Dura...","[Bar_73, Time Signature_4/4, Instrument_Electr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10737,TRHCJMM128F933BEED,"[Bar_109, Time Signature_4/4, Note Density_3, ...",0.721,0.672,6,-9.029,0,0.0547,0.010600,0.804,104.976,4,"[Mean Pitch_12, Mean Pitch_13, Mean Pitch_12, ...","[Mean Velocity_23, Mean Velocity_24, Mean Velo...","[Note Density_3, Note Density_4, Note Density_...","[Mean Duration_29, Mean Duration_28, Mean Dura...","[Bar_109, Time Signature_4/4, Instrument_Slap ..."
10738,TRHCJMM128F933BEED,"[Bar_109, Time Signature_4/4, Note Density_3, ...",0.706,0.747,6,-6.336,0,0.0724,0.001150,0.800,104.960,4,"[Mean Pitch_12, Mean Pitch_13, Mean Pitch_12, ...","[Mean Velocity_23, Mean Velocity_24, Mean Velo...","[Note Density_3, Note Density_4, Note Density_...","[Mean Duration_29, Mean Duration_28, Mean Dura...","[Bar_109, Time Signature_4/4, Instrument_Slap ..."
10739,TRLTOOL128F92CFC61,"[Bar_97, Time Signature_4/4, Note Density_2, M...",0.442,0.885,4,-5.843,1,0.0810,0.000000,0.268,129.421,4,"[Mean Pitch_12, Mean Pitch_12, Mean Pitch_12, ...","[Mean Velocity_32, Mean Velocity_32, Mean Velo...","[Note Density_2, Note Density_2, Note Density_...","[Mean Duration_27, Mean Duration_25, Mean Dura...","[Bar_97, Time Signature_4/4, Instrument_Electr..."
10740,TRAUWUX128F92FE41C,"[Bar_49, Time Signature_4/4, Note Density_3, M...",0.514,0.243,3,-12.348,1,0.7520,0.000540,0.388,171.816,4,"[Mean Pitch_13, Mean Pitch_13, Mean Pitch_13, ...","[Mean Velocity_19, Mean Velocity_20, Mean Velo...","[Note Density_3, Note Density_3, Note Density_...","[Mean Duration_32, Mean Duration_32, Mean Dura...","[Bar_49, Time Signature_4/4, Instrument_Bright..."


In [ ]:
df_merged.to_csv("processed_dataset.csv")

## Train an LSTM on the whole description

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# # Build vocabulary
# token_list = set(token for desc in df_merged['description'] for token in desc)
# vocab = {token: i for i, token in enumerate(token_list, start=1)}
# vocab['<PAD>'] = 0  # Add a padding token

# print(len(vocab))

In [ ]:
# MUSIC_FEATURES = ['danceability', 'energy', 'key',
#                   'loudness', 'mode', 'acousticness',
#                   'instrumentalness', 'valence',
#                   'tempo', 'time_signature']

# # Data preparation
# class MusicDataset(Dataset):
#     def __init__(self, dataframe):
#         features = []
#         for i in range(len(dataframe.index)):
#           row = []
#           for feature_name in MUSIC_FEATURES:
#             row.append(dataframe.iloc[i][feature_name])
#           features.append(row)
#         self.features = features
#         self.descriptions = dataframe['description']

#     def __len__(self):
#         return len(self.descriptions)

#     def __getitem__(self, idx):
#         feature = torch.tensor(self.features[idx], dtype=torch.float32)
#         description = [vocab[token] for token in self.descriptions[idx]]
#         description_tensor = torch.tensor(description, dtype=torch.long).to(device)
#         return feature, description_tensor

In [ ]:
MUSIC_FEATURES = ['danceability', 'energy', 'key',
                  'loudness', 'mode', 'acousticness',
                  'instrumentalness', 'valence',
                  'tempo', 'time_signature']

# Data preparation
class MusicDataset(Dataset):
    def __init__(self, dataframe, description_column, numerical_feature=False):
        features = []
        for i in range(len(dataframe.index)):
          row = []
          for feature_name in MUSIC_FEATURES:
            row.append(dataframe.iloc[i][feature_name])
          features.append(row)
        self.features = features
        self.descriptions = dataframe[description_column]
        self.numerical = numerical_feature

        if not self.numerical:
          token_list = set(token for desc in df_merged[description_column] for token in desc)
          self.vocab = {token: i for i, token in enumerate(token_list, start=1)}
          self.vocab['<PAD>'] = 0  # Add a padding token

          print(f"Vocab size for {description_column}", len(self.vocab))

    def __len__(self):
        return len(self.descriptions)

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        if self.numerical:
          description = [int(x.split('_')[-1]) for x in self.descriptions[idx]]
        else:
          description = [self.vocab[token] for token in self.descriptions[idx]]
        description_tensor = torch.tensor(description + [0]*(max_description_length - len(description)), dtype=torch.long)
        return feature.to(device), description_tensor.to(device)

In [ ]:
# pitch_dataset = MusicDataset(df_merged, 'pitch_description', numerical_feature=True)
# density_dataset = MusicDataset(df_merged, 'density_description', numerical_feature=True)
# duration_dataset = MusicDataset(df_merged, 'duration_description', numerical_feature=True)
# velocity_dataset = MusicDataset(df_merged, 'velocity_description', numerical_feature=True)


In [ ]:
full_dataset = MusicDataset(df_merged, 'description', numerical_feature=False)

Vocab size for description 633


## Transformer

In [ ]:
loader = DataLoader(full_dataset, batch_size=192, shuffle=True)


# Model configuration
d_model = 16  # Model dimensionality
nhead = 8      # Number of heads in multi-head attention
num_layers = 6 # Number of transformer layers
vocab_size = len(full_dataset.vocab) # Vocabulary size

class CustomTransformer(nn.Module):
    def __init__(self, vocab_size=vocab_size):
        super().__init__()
        self.feature_embedding = nn.Linear(10, d_model)  # Assuming 10 numerical features
        self.token_embedding = nn.Embedding(vocab_size, d_model, padding_idx=full_dataset.vocab['<PAD>'])
        self.positional_encoding = nn.Parameter(torch.zeros(1, 512, d_model)) # Example positional encoding, adjust size as needed
        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead),
            num_layers=num_layers
        )
        self.output_layer = nn.Linear(d_model, vocab_size)

    def forward(self, src, tgt):
        src = self.feature_embedding(src).unsqueeze(1) # Transform feature to match input embedding size
        tgt = self.token_embedding(tgt) + self.positional_encoding[:,:tgt.size(1),:] # Add positional encoding
        output = self.decoder(tgt, src.repeat(1, tgt.size(1), 1))
        logits = self.output_layer(output)
        return logits

In [ ]:
import datetime
# Initialize model
model = CustomTransformer().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Training loop
for epoch in range(5):
    print("Epoch", epoch+1)
    epoch_losses = []
    start = datetime.datetime.utcnow()
    for feature, description in loader:
        optimizer.zero_grad()
        # Prepare shifted target input and labels for training
        input_tgt = torch.cat([torch.full((description.shape[0], 1), full_dataset.vocab['<PAD>'], dtype=torch.long), description[:, :-1].cpu()], dim=1).to(device)
        logits = model(feature, input_tgt)
        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(logits.view(-1, vocab_size), description.view(-1))
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
        # print(f'Epoch {epoch}, Loss: {loss.item()}')
    print(f"Average loss {np.mean(epoch_losses):.2f}")
    print(f"Seconds per epoch", (datetime.datetime.utcnow() - start).total_seconds())

torch.save(model, "transformer.pt")

Epoch 1
Average loss 5.96
Seconds per epoch 44.051224
Epoch 2
Average loss 5.53
Seconds per epoch 42.498491
Epoch 3
Average loss 5.42
Seconds per epoch 42.668151
Epoch 4
Average loss 5.31
Seconds per epoch 42.726678
Epoch 5
Average loss 5.20
Seconds per epoch 42.718076


In [ ]:
# LSTM Model Configuration
class LSTMModel(nn.Module):
    def __init__(self, vocab_size=vocab_size):
        super().__init__()
        self.embedding_dim = 128
        self.hidden_dim = 256
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(self.embedding_dim + 10, self.hidden_dim, batch_first=True)  # +10 for concatenated features
        self.fc = nn.Linear(self.hidden_dim, self.vocab_size)

    def forward(self, features, descriptions):
        embeddings = self.embedding(descriptions)
        # Concatenate features to each input embedding
        features_expanded = features.unsqueeze(1).repeat(1, descriptions.size(1), 1)
        lstm_input = torch.cat((embeddings, features_expanded), dim=2)
        lstm_out, _ = self.lstm(lstm_input)
        logits = self.fc(lstm_out)
        return logits

In [ ]:


# # Initialize model
# model = LSTMModel()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# # Training loop
# for epoch in range(10):  # You might need more epochs
#     for feature, description in loader:
#         optimizer.zero_grad()
#         # Prepare input and labels for training
#         input_desc = torch.cat([torch.full((description.shape[0], 1), vocab['<PAD>'], dtype=torch.long), description[:, :-1]], dim=1)
#         logits = model(feature, input_desc)
#         loss_fn = nn.CrossEntropyLoss()
#         loss = loss_fn(logits.view(-1, len(vocab)), description.view(-1))
#         loss.backward()
#         optimizer.step()
#         print(f'Epoch {epoch}, Loss: {loss.item()}')

## Predicting token classes separately

In [ ]:
pitch_dataset = MusicDataset(df_merged, 'pitch_description', numerical_feature=True)
density_dataset = MusicDataset(df_merged, 'density_description', numerical_feature=True)
duration_dataset = MusicDataset(df_merged, 'duration_description', numerical_feature=True)
velocity_dataset = MusicDataset(df_merged, 'velocity_description', numerical_feature=True)
categorical_dataset = MusicDataset(df_merged, 'categorical_description', numerical_feature=False)

Vocab size for categorical_description 527


In [ ]:
import datetime

def train_on_subset(dataset, numerical=False):

    loader = DataLoader(dataset, batch_size=1, shuffle=True)
    # Initialize model

    model_vocab_size = 1 if numerical else len(dataset.vocab)
    model = LSTMModel(model_vocab_size).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    if numerical:
      loss_fn = nn.MSELoss()
    else:
      loss_fn = nn.CrossEntropyLoss()
    # Training loop
    losses = []
    for epoch in range(10):  # You might need more epochs
        print("Epoch", epoch+1)
        epoch_losses = []
        start =datetime.datetime.utcnow()
        for feature, description in loader:
            optimizer.zero_grad()
            # Prepare input and labels for training
            input_desc = torch.cat([torch.full((description.shape[0], 1), 0, dtype=torch.long), description[:, :-1].cpu()], dim=1).to(device)
            logits = model(feature, input_desc)

            target = description.view(-1)
            if numerical: target = target.float()
            loss = loss_fn(logits.view(-1, model_vocab_size), target)
            loss.backward()
            optimizer.step()
            epoch_losses.append(loss.item())
            # print(f'Epoch {epoch}, Loss: {loss.item()}', end="\r", flush=True)
        print(f"Average epoch loss: {np.mean(epoch_losses):.2f}")
        print(f"Seconds per ep: {(datetime.datetime.utcnow() - start).total_seconds()}")
        losses += epoch_losses

    return losses, model

In [ ]:
losses, model = train_on_subset(pitch_dataset, numerical=True)
torch.save(model, "pitch_lstm.pt")

losses, model = train_on_subset(velocity_dataset, numerical=True)
torch.save(model, "velocity_lstm.pt")

losses, model = train_on_subset(duration_dataset, numerical=True)
torch.save(model, "duration_lstm.pt")

losses, model = train_on_subset(density_dataset, numerical=True)
torch.save(model, "density_lstm.pt")

losses, model = train_on_subset(categorical_dataset, numerical=False)
torch.save(model, "categorical_lstm.pt")

Epoch 1


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
losses, model = train_on_subset(full_dataset, numerical=False)
torch.save(model, 'full_lstm.pt')

Epoch 1
Average epoch loss: 2.50
Epoch 2
Average epoch loss: 1.96
Epoch 3
Average epoch loss: 1.73
Epoch 4
Average epoch loss: 1.55
Epoch 5
Average epoch loss: 1.43
Epoch 6
Average epoch loss: 1.35
Epoch 7
Average epoch loss: 1.29
Epoch 8
Average epoch loss: 1.24
Epoch 9
Average epoch loss: 1.19
Epoch 10
Average epoch loss: 1.15
